<a href="https://colab.research.google.com/github/parsa-ghofrani/DataColab_entrance/blob/main/DataColab_entrance_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files


uploaded = files.upload()

Saving data.zip to data.zip


In [ ]:
!unzip /content/data.zip -d /content/multiple_choice_data

Archive:  /content/data.zip
   creating: /content/multiple_choice_data/data/
  inflating: /content/multiple_choice_data/data/train_data.csv  
  inflating: /content/multiple_choice_data/data/test_data.csv  


In [ ]:
!pip uninstall -y jax jaxlib


Found existing installation: jax 0.5.2
Uninstalling jax-0.5.2:
  Successfully uninstalled jax-0.5.2
Found existing installation: jaxlib 0.5.1
Uninstalling jaxlib-0.5.1:
  Successfully uninstalled jaxlib-0.5.1


In [ ]:
pip install jax


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 71.6 MB/s eta 0:00:00
  Attempting uninstall: ml_dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires ml-dtypes<0.5.0,>=0.4.0, but you have ml-dtypes 0.5.1 which is incompatible.


In [ ]:
from transformers import AutoTokenizer,AutoModelForCausalLM, pipeline

model_identification = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_identification)
model = AutoModelForCausalLM.from_pretrained(model_identification, device_map="auto")

qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=a06a1d734bf72074133b3c8bd81c5b2a9ac0c9588850791cf4b2e6d4b3b8164a
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [ ]:
import wikipedia

def retrieve_wikipedia_content(question, max_chars=1000):
    try:
        search_results = wikipedia.search(question)
        if not search_results:
            return "There was no relevant context!"

        page = wikipedia.page(search_results[0])
        content = page.content

        return content[:max_chars]

    except Exception as e:
        return f"There was an error retrieving the documents. Error message: {e}"


sample_question = "What is the second law of thermodynamics?"
context = retrieve_wikipedia_content(sample_question)
print(context)


The second law of thermodynamics is a physical law based on universal empirical observation concerning heat and energy interconversions. A simple statement of the law is that heat always flows spontaneously from hotter to colder regions of matter (or 'downhill' in terms of the temperature gradient). Another statement is: "Not all heat can be converted into work in a cyclic process."
The second law of thermodynamics establishes the concept of entropy as a physical property of a thermodynamic system. It  predicts whether processes are forbidden despite obeying the requirement of conservation of energy as expressed in the first law of thermodynamics and provides necessary criteria for spontaneous processes. For example, the first law allows the process of a cup falling off a table and breaking on the floor, as well as allowing the reverse process of the cup fragments coming back together and 'jumping' back onto the table, while the second law allows the former and denies the latter. The s

In [ ]:
import random

def get_few_shot_examples(training_df, k=5):
    return training_df.sample(n=k).reset_index(drop=True)



In [ ]:
def build_prompt(wiki_context, few_shot_examples, test_question_row):
    prompt = f"Context:\n{wiki_context.strip()}\n\n"

    # Add few-shot training examples
    for _, row in few_shot_examples.iterrows():
        prompt += f"Q: {row['prompt']}\n"
        for opt in ['A', 'B', 'C', 'D', 'E']:
            prompt += f"{opt}. {row[opt]}\n"
        prompt += f"Answer: {row['answer']}\n\n"

    # Add the actual test question
    prompt += f"Q: {test_question_row['prompt']}\n"
    for opt in ['A', 'B', 'C', 'D', 'E']:
        prompt += f"{opt}. {test_question_row[opt]}\n"
    prompt += "Answer:"

    return prompt


In [ ]:
import pandas as pd

df = pd.read_csv("multiple_choice_data/data/train_data.csv")
few_shot = get_few_shot_examples(df, k=5)
test_question = {
    "prompt": "Which gas is most abundant in Earth's atmosphere?",
    "A": "Oxygen",
    "B": "Carbon Dioxide",
    "C": "Nitrogen",
    "D": "Hydrogen",
    "E": "Argon"
}

# Then retrieve the Wikipedia context
wiki_context = retrieve_wikipedia_content(test_question["prompt"])
prompt = build_prompt(wiki_context, few_shot, test_question)
print(prompt)

Context:
There was an error retrieving the documents. Error message: Page id "atmosphere on earth" does not match any pages. Try another id!

Q: What is the isophotal diameter used for in measuring a galaxy's size?
A. The isophotal diameter is a way of measuring a galaxy's distance from Earth.
B. The isophotal diameter is a measure of a galaxy's age.
C. The isophotal diameter is a measure of a galaxy's mass.
D. The isophotal diameter is a measure of a galaxy's temperature.
E. The isophotal diameter is a conventional way of measuring a galaxy's size based on its apparent surface brightness.
Answer: E

Q: What is the Peierls bracket in canonical quantization?
A. The Peierls bracket is a mathematical symbol used to represent the Poisson algebra in the canonical quantization method.
B. The Peierls bracket is a mathematical tool used to generate the Hamiltonian in the canonical quantization method.
C. The Peierls bracket is a Poisson bracket derived from the action in the canonical quantiza

In [ ]:
import pandas as pd
df.head()

,prompt,A,B,C,D,E,answer
0,What is the main sequence in astronomy?,The main sequence is a type of galaxy that con...,The main sequence is a type of black hole that...,The main sequence is a continuous and distinct...,The main sequence is a group of planets that o...,The main sequence is a type of nebula that is ...,C
1,"What is the ""ultraviolet catastrophe""?",It is a phenomenon that occurs only in multi-m...,It is the misbehavior of a formula for higher ...,It is the standing wave of a string in harmoni...,It is a flaw in classical physics that results...,It is a disproven theory about the distributio...,B
2,What is the Carnot engine?,The Carnot engine is a theoretical engine that...,The Carnot engine is an ideal heat engine that...,The Carnot engine is a real heat engine that o...,The Carnot engine is a theoretical engine that...,The Carnot engine is a real engine that operat...,B
3,What is accelerator-based light-ion fusion?,Accelerator-based light-ion fusion is a techni...,Accelerator-based light-ion fusion is a techni...,Accelerator-based light-ion fusion is a techni...,Accelerator-based light-ion fusion is a techni...,Accelerator-based light-ion fusion is a techni...,A
4,"What is a ""coffee ring"" in physics?",A type of coffee that is made by boiling coffe...,A pattern left by a particle-laden liquid afte...,A type of coffee that is made by mixing instan...,A type of coffee that is made by pouring hot w...,A pattern left by a particle-laden liquid afte...,E


In [ ]:
def get_answer_from_model(prompt):
    output = qa_pipeline(prompt, max_new_tokens=5, temperature=0.0)[0]['generated_text']

    # Extract only the final answer letter after "Answer:"
    try:
        answer = output.split("Answer:")[-1].strip()
        # Keep only the first uppercase letter
        answer = next((ch for ch in answer if ch in "ABCDE"), "A")  # Default to A if unclear
        return answer
    except:
        return "couldm't"




wiki_context = retrieve_wikipedia_content(test_question["prompt"])
few_shot = get_few_shot_examples(df, k=5)
prompt = build_prompt(wiki_context, few_shot, test_question)

predicted_answer = get_answer_from_model(prompt)
print("Predicted Answer:", predicted_answer)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Predicted Answer: C


In [ ]:
correct = 0
total = 0
predictions = []

for i, row in df.iterrows():
    test_question = {
        "prompt": row["prompt"],
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"],
        "E": row["E"]
    }

    real_answer = row["answer"]

    # RAG: Get Wikipedia context
    wiki_context = retrieve_wikipedia_content(test_question["prompt"])

    # Few-shot: Randomly sample 5 different examples (excluding the current one)
    few_shot = df.drop(i).sample(n=5).reset_index(drop=True)

    # Prompt
    prompt = build_prompt(wiki_context, few_shot, test_question)

    # LLM answer
    predicted_answer = get_answer_from_model(prompt)

    # Store and compare
    predictions.append(predicted_answer)
    if predicted_answer == real_answer:
        correct += 1
    total += 1

    print(f"Q{i+1}: Predicted = {predicted_answer}, Actual = {real_answer}")

accuracy = correct / total
print(f"\nTraining Set Accuracy: {accuracy:.2%}")


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q1: Predicted = C, Actual = C


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q2: Predicted = B, Actual = B


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Q48: Predicted = E, Actual = E


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q49: Predicted = A, Actual = A


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q50: Predicted = B, Actual = D

Training Set Accuracy: 70.00%


In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype="auto")

qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2.
401 Client Error. (Request ID: Root=1-6815dad3-5ee69b273c0baf8b6d29ac9b;b86a801e-af3d-4e66-b62c-1ea3443554a6)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.2 is restricted. You must have access to it and be authenticated to access it. Please log in.